Some common headers

In [1]:
#include <exception> // std::runtime_error
#include <iostream> // std::cout
#include <memory> // smart pointers
#include <string> // std::string
#pragma clang diagnostic push
#pragma clang diagnostic ignored "-Wnonnull" // allow to pass nullptrs

## 1. Example of problem with manual memory management

In [2]:
{
struct Foo{
    void calculate() {
        throw std::runtime_error("oopps, sth goes wrong");
    }
    std::string result() {return "result of Foo";}
};

Foo* foo = new Foo();
foo->calculate();
std::cout << foo->result() << std::endl;
delete foo;
}

Standard Exception: oopps, sth goes wrong

<span style="color:red">**Memory handled by foo is never released - memory leak**

### 1a. we can of caurse try to catch an exception

In [3]:
{
struct Foo{
    void calculate() {
        throw std::runtime_error("oopps, sth goes wrong");
    }
    std::string result() {return "result of Foo";}
};

Foo* foo = new Foo();
try {
    foo->calculate();
} catch (...) {
    delete foo;
    return;
}
std::cout << foo->result() << std::endl;
delete foo;
}

- it causes code duplication
- the code is more unreadable
- very often it's hard to handle all possible paths
- some problems can be resolved by custom RAII impl (resource acqusition is initialization)

## 2. Smart pointers:
- memory management of objects allocated on heap
- responsible for calling delete
- clear ownership
- overloaded opeartor* and operator-> provide usage similar to raw pointers

### 2a. Unique Pointer

In [5]:
{
struct Foo{
    void calculate() {
        throw std::runtime_error("oopps, sth goes wrong");
    }
    std::string result() {return "result of Foo";}
};

auto foo = std::make_unique<Foo>();
foo->calculate();
std::cout << foo->result() << std::endl;
} // Foo memmory is released despite exception

Standard Exception: oopps, sth goes wrong

- unique pointer cannot be copied or assigned, only moving is allowed
- it means that allocated object can be pointed by **only one unique_ptr**

In [6]:
struct A {
    int calculate() {return 44;}
};

In [7]:
void foo_processing(std::unique_ptr<A> a) {
    a->calculate();
}

auto a = std::make_unique<A>();
foo_processing(std::move(a));
std::cout << a->calculate() << std::endl; // a is nullptr in this moment, the exception is thrown

Interpreter Exception: 

so it should be (but rather doesn't make a sense):

In [8]:
std::unique_ptr<A> foo_processing(std::unique_ptr<A> a) {
    a->calculate();
    return std::move(a);
}

auto a = std::make_unique<A>();
a = foo_processing(std::move(a));
std::cout << a->calculate() << std::endl;

44


<span style="color:blue">Note that the smart pointer with base class type can point to the derived class object (similar to raw pointers/references)</span>

In [9]:
class Base {
    public:
    virtual void hello() {std::cout << "I am from base class \n"; }
    virtual ~Base() = default;
};
class Derived : public Base {
    public:
    void hello() override {std::cout << "I am from derived class \n"; }
};

std::unique_ptr<Base> b = std::make_unique<Derived>();
b->hello();

I am from derived class 


## 2b. shared pointers:
We can have more than one pointers to the same allocated object.

It's based on reference counters: 
- ctor initialize counter with 1
- copy ctor, copy assignment operator increment counter
- dtor decrement couter
- if couter == 0, the memory is released

In [10]:
void foo_processing_shared(std::shared_ptr<A> a) { // counter=2
    a->calculate();
} // counter=1

{
auto a = std::make_shared<A>(); // counter=1
foo_processing_shared(a);
std::cout << a->calculate() << std::endl;
} // counter=0, mememory is released

44


In [ ]:
Note that shared_pointer is heavier and slower than unique_ptr